In [36]:
import pandas as pd


path = '/Users/arianiherrera/Desktop/NYCDataScience/InstaPredict_Capstone/image_posts_df.csv'

insta_combined_df = pd.read_csv(path)
insta_combined_df.shape

(62537, 50)

In [37]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [41]:
# features to drop from training set
feats_to_drop = ['Unnamed: 0_x', 'X1', 'caption', 'image', 'key', 'new_id', 'photo_id',
                  'scrape_time', 'user', 'index', 'image_key', 'post_time', 'time', 'likes'
                 ]
insta_combined_df = insta_combined_df.loc[insta_combined_df['tag'] == 'nature']
# use only nature tage for now
X = insta_combined_df.drop(feats_to_drop, axis=1)
y = insta_combined_df[['likes']]

In [42]:
print X.shape
print y.shape

(46438, 36)
(46438, 1)


In [48]:
X.drop('tag', axis=1, inplace = True)

In [43]:
# need to edit engineering file
from sklearn.preprocessing import LabelEncoder
from textblob import TextBlob
le = LabelEncoder()
X['filters'] = le.fit_transform(X['filters'])

In [44]:
X['num_hashtags'] = X.hashtags.apply(lambda x: len(x))
X['hashtags'] = X.hashtags.apply(lambda x: ''.join(x))
X['hashtag_polarity'] = X.hashtags.apply(lambda x: TextBlob(x).sentiment.polarity)
X['hashtag_subjectivity'] = X.hashtags.apply(lambda x: TextBlob(x).sentiment.subjectivity)


In [45]:
X.drop('hashtags', axis=1, inplace = True)

In [46]:
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
print X.shape
print y.shape

(46438, 38)
(46438, 1)


In [66]:
X.head()

,Unnamed: 0,comments,filters,B_max,B_mean,B_median,B_rms,B_stddev,B_sum,B_sum2,...,Unnamed: 0_y,blur,image_height,image_width,luminance,num_faces,percieved_luminance,num_hashtags,hashtag_polarity,hashtag_subjectivity
0,0,2.0,26,255,98.992234,82,113.167626,54.840215,40547219.0,5.245711e+09,...,0.0,868.595552,640,640,4.002933e+07,1,5.477206e+09,300,0.266667,0.544444
1,1,0.0,11,251,105.343421,90,118.525219,54.323025,37822502.0,5.043876e+09,...,0.0,1356.346593,561,640,4.266245e+07,0,6.321252e+09,41,-0.200000,0.300000
2,2,0.0,26,255,71.965190,67,88.129805,50.871149,29476942.0,3.181307e+09,...,0.0,727.699482,640,640,3.396947e+07,1,4.007398e+09,97,0.000000,0.000000
3,3,3.0,12,255,138.993707,119,149.381904,54.733012,35671345.0,5.726910e+09,...,0.0,1730.119484,401,640,3.484327e+07,0,5.994941e+09,133,0.000000,0.000000
4,4,0.0,26,255,93.235804,80,117.934467,72.219272,28642039.0,4.272703e+09,...,0.0,4770.569102,480,640,3.563972e+07,0,5.866979e+09,217,0.316667,0.466667


In [62]:
from keras.layers.embeddings import Embedding

# create baseline model
def baseline_model():
    
    model = Sequential()
    model.add(Dense(37, input_dim = 37, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model
# Standardized Results: 188632.41 (231428.25) MSE


# model with more depth
def deeper_model():
    
    model = Sequential()
    model.add(Embedding(X.shape[0], 128, input_length=37))
    model.add(LSTM(128))
    model.add(Dropout(.4))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model
# Standardized Results: 188732.72 (231650.47) MSE

# model with more width
def wider_model():
    
    model = Sequential()
    model.add(Dense(76, input_dim = 37, init='normal', activation='relu'))
    model.add(Dense(1, init='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model
# Standardized Results: 188600.85 (231454.54) MSE    
    
    
# fix random seed
seed = 0
np.random.seed(seed)

# create estimators with standardized data set
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=deeper_model, nb_epoch = 50, batch_size = 100, verbose=2)))

# perform 10-fold cross validation using pipeline to ensure that there is no data leakage
# in order to evaluate model
pipeline = Pipeline(estimators)
kfold = KFold(n=X.shape[0], n_folds=5, shuffle = True, random_state = seed)
results = cross_val_score(pipeline, X.as_matrix(), y.as_matrix(), cv=kfold)
print("Standardized Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

TypeError: Expected int32, got list containing Tensors of type '_Message' instead.

Try new approach with user data combined

In [67]:
user_data = pd.read_csv('/Users/arianiherrera/Downloads/users.csv')

/Users/arianiherrera/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
user_data.head()

,Unnamed: 0,Followers,Following,Posts,Unnamed: 0.1,bio,full_name,list_likes,max_likes,mean_likes,median_likes,min_likes,scrape_time,top_N_dic,top_N_photos,user
0,0,72.0,99.0,76.0,0.0,– 📷nikond5000\n❤collectmomentswithmynikon\n🗺...,Photos of Travel and Moments🌍,"[28, 30, 22, 14, 14, 16, 15, 26, 12, 15, 14, 2...",30,17.85,15.5,9,2017-03-23 14:12:24.954712,{'image': [u'https://scontent.cdninstagram.com...,"{'comments': [4, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0,...",nb_photosoflife
1,1,180.0,254.0,112.0,1.0,NaN,Irene 🌹,"[45, 45, 24, 41, 48, 42, 14, 6, 12, 16, 11, 15...",48,22.85,17.0,6,2017-03-23 14:12:24.986175,{'image': [u'https://scontent.cdninstagram.com...,"{'comments': [1, 2, 0, 2, 7, 0, 0, 0, 3, 1, 2,...",irene_life8
2,2,61.0,75.0,65.0,2.0,– 24. London (LGW). Flight Attendant. Dreamer....,NaN,"[53, 56, 36, 23, 32, 43, 29, 41, 50, 32, 28, 8...",81,41.30,40.5,23,2017-03-23 14:12:25.007444,{'image': [u'https://scontent.cdninstagram.com...,"{'comments': [2, 2, 2, 1, 4, 4, 1, 1, 0, 0, 1,...",mynameisevgeni
3,3,352.0,985.0,614.0,3.0,"– ""Partiaaaal""",Victor,"[29, 28, 25, 17, 11, 12, 21, 139, 29, 40, 38, ...",139,38.80,33.0,11,2017-03-23 14:12:25.035681,{'image': [u'https://scontent.cdninstagram.com...,"{'comments': [1, 0, 2, 0, 1, 2, 0, 1, 0, 2, 0,...",victorfs85
4,4,600.0,531.0,1161.0,4.0,– Demonic_daddy666 is my other account \nPagan...,Andy Blaire,"[43, 40, 45, 46, 25, 21, 38, 17, 40, 36, 31, 3...",55,34.45,38.0,6,2017-03-23 14:12:25.112356,{'image': [u'https://scontent.cdninstagram.com...,"{'comments': [0, 1, 2, 0, 0, 0, 1, 0, 1, 0, 0,...",daddy_reaper666
